In [1]:
import numpy as np

In [16]:
n_palabras_query = 2
tipos_query = ["Artista y Letra","Letra"]
tipo_query = tipos_query[0]

canciones = ["Hola yo me llamo pepito","Bum Bum la rumba se prende"]

for c in canciones:
    ## TEMPORAL ##
    tokens = c.split(" ")
    ## TEMPORAL ##
    



me llamo
Bum Bum


## Implementaciones

### IR - TF-IDF

In [21]:
import sklearn.feature_extraction.text
import sklearn.metrics.pairwise
import pandas
import numpy as np
import plotly.express as px
import spacy
import simplemma
import random
model_es = spacy.load("es_core_news_sm")
def reemplazar_tildes(texto: str) -> str:
    reemplazos = {"á": "a", "é": "e", "í": "i", "ó": "o", "ú": "u"}
    for original, reemplazo in reemplazos.items():
        texto = texto.replace(original, reemplazo)
    return texto
def normalize_text(text: str) -> str:
    doc = model_es(" ".join(simplemma.text_lemmatizer(text, lang="es")))
    return " ".join([reemplazar_tildes(str(token)) for token in doc if token.is_alpha and not token.is_stop])
df_normalize = pandas.read_csv("./data_normalize.csv")
corpus = df_normalize["normalize"].to_list()
vectorizer = sklearn.feature_extraction.text.TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
def process_query(query: str) -> str:
    return normalize_text(query)
def get_query_vector(processed_query: str) -> np.ndarray:
    query_vector = vectorizer.transform([processed_query]).toarray()
    return query_vector
def get_song_recommendations(query_vector: np.ndarray, top_n=5) -> pandas.DataFrame:
    cosine_similarities = sklearn.metrics.pairwise.cosine_similarity(query_vector, X).flatten()
    related_docs_indices = cosine_similarities.argsort()[: -top_n - 1 : -1]
    return df_normalize.iloc[related_docs_indices]
def search_and_recommend(query: str, top_n=5) -> pandas.DataFrame:
    processed_query = process_query(query)
    query_vector = get_query_vector(processed_query)
    recommendations = get_song_recommendations(query_vector, top_n)
    return recommendations


### IR - WORD2VEC

In [18]:
import nltk
nltk.download("all", quiet=True)
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import gensim
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors
import scipy
from scipy import spatial
import pandas as pd
wordvectors_file_vec = "/embeddings-l-model.vec"
# cantidad = 100000
model = KeyedVectors.load_word2vec_format(wordvectors_file_vec)  # , limit=cantidad)
ruta = "../../../data/data.csv"
data = pd.read_csv(ruta)
data["unido_todo"] = (
    ((data["Artista"] + " ") * 5)
    + ((data["Titulo"] + " ") * 5)
    + data["Cancion"]  # Se multiplica por 5 para darle un mayor peso a artista y titulo
)
stop_words = set(stopwords.words("spanish"))
spanishstemmer = SnowballStemmer("spanish")
def preprocesamiento_stemas(text: str) -> pd.DataFrame:
    """
    Función encargada de prepocesar la data de las canciones
    """
    text = str(text)
    text = text.lower()
    text = "".join([word for word in text if word not in string.punctuation])
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    stema = [spanishstemmer.stem(w) for w in tokens]  # devuelve palabras stemizadas
    return " ".join(stema)
X = data["unido_todo"].apply(preprocesamiento_stemas)
X.head()
def embeddings(word: str) -> np.array:
    """
    Función encargada de realizar los embeddings de las palabras
    """
    if word in model.key_to_index:
        return model.get_vector(word)
    else:
        return np.zeros(300)
diccionario = {}
for idx, fila in enumerate(X):
    average_vector = np.mean(
        np.array([embeddings(palabra) for palabra in fila.split()]), axis=0
    )
    d1 = {idx: (average_vector)}
    diccionario.update(d1)
def similaridad(query_embedding: np.array, average_vec: np.array) -> float:
    """
    Calcular la similitud del coseno
    """
    sim = [(1 - scipy.spatial.distance.cosine(query_embedding, average_vec))]
    return sim
def seleccion_canciones_word2vec(query: str) -> list:
    """
    Función que calcula distancias entre query y canciones.
    """
    query_words = np.mean(
        np.array(
            [embeddings(palabra) for palabra in preprocesamiento_stemas(query).split()],
            dtype=float,
        ),
        axis=0,
    )
    rank = []
    for k, v in diccionario.items():
        rank.append((k, similaridad(query_words, v)))  # data['Titulo'][k]
    rank = sorted(rank, key=lambda t: t[1], reverse=True)
    print("Canciones relacionadas: ")
    return rank[:20]
def main(texto: str) -> pd.DataFrame:
    """
    Función principal para encontrar la canción solicitada
    """
    canciones_respuesta = seleccion_canciones_word2vec(texto)
    diccionario_resultado = {"Titulo": [], "Artista": [], "Simularidad": []}
    for cancion in canciones_respuesta:
        diccionario_resultado["Titulo"].append(data["Titulo"][cancion[0]])
        diccionario_resultado["Artista"].append(data["Artista"][cancion[0]])
        diccionario_resultado["Simularidad"].append(cancion[1])
    return pd.DataFrame(diccionario_resultado)

KeyboardInterrupt: 

## Evaluación

In [176]:
from simplemma import simple_tokenizer
def get_tokens(text: str) -> str:
    doc = model_es(" ".join(simple_tokenizer(text)))
    return [reemplazar_tildes(str(token)) for token in doc if token.is_alpha and not token.is_stop]

def get_random_query(cancion,n_palabras_query=5):
    tokens = get_tokens(cancion)
    longitud_cancion = len(tokens)
    try:
        ultimo_token_query = max(longitud_cancion - n_palabras_query,1)
        # print(ultimo_token_query)
        pos_query = np.random.randint(0,ultimo_token_query)
        # print(pos_query)
        query = " ".join(tokens[pos_query:min(pos_query+n_palabras_query,len(tokens))])
        return query
    except:
        print('Exception')
        return "lasdlasd"
  
def resultado_prediccion(row,top_n=5):
    letra_cancion = row['Cancion']
    target_idx = row.name
    query = get_random_query(letra_cancion)
    predicted_idxs = search_and_recommend(query,top_n=top_n).index.tolist()
    return int(target_idx in predicted_idxs), target_idx, predicted_idxs, query

def evaluar(df,tamaño_muestra=100):
    sample_idxs = np.random.choice(df.index.tolist(),size=tamaño_muestra)
    df_eval = df.loc[sample_idxs,:]
    info = df_eval.apply(resultado_prediccion,top_n=5,axis=1)
    res = np.array([x for x,y,z,a in info])
    target_idx = [y for x,y,z,a in info]
    predicted_idxs = [z for x,y,z,a in info]
    query = [a for x,y,z,a in info]
    return res.mean(), res, target_idx, predicted_idxs, query

In [179]:
accs = []
resultados = []
ti = []
pi = []
q = []
for i in range(20):
    acc, res, target_idx, predicted_idxs, query = evaluar(df_normalize,tamaño_muestra=200)
    accs.append(acc)
    resultados.append(res)
    ti.append(target_idx)
    pi.append(predicted_idxs)
    q.append(query)
    
    print("E"+str(i+1)+"d-",end="")
print("")
acc_mean = np.array(accs).mean()
print("Accuracy Promedio = "+str(np.round(acc_mean,4)))

E1d-E2d-E3d-E4d-E5d-E6d-E7d-E8d-E9d-E10d-E11d-E12d-E13d-E14d-E15d-E16d-E17d-E18d-E19d-E20d-
Accuracy Promedio = 0.6478


In [178]:
ej = 10
print(ti[0][ej])
print(pi[0][ej])
print(q[0][ej])

322
[2184, 634, 2265, 1139, 623]
vida mundo acabara imposible existir
